In [1]:
# Importamos las bibliotecas necesarias
import os
import re
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import Word2Vec, KeyedVectors
import seaborn as sns

In [2]:
# Fijamos semilla para reproducibilidad
np.random.seed(42)
import random
random.seed(42)

In [3]:
# Descargamos recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/aquiles/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aquiles/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/aquiles/nltk_data...


True

In [4]:
def listar_archivos(ruta_base):
    """
    Lista todos los archivos en las subcarpetas de la ruta base
    """
    archivos = []
    categorias = []
    
    # Recorremos todas las subcarpetas (categorías)
    for categoria in os.listdir(ruta_base):
        ruta_categoria = os.path.join(ruta_base, categoria)
        
        # Verificamos que sea un directorio
        if os.path.isdir(ruta_categoria):
            # Listamos los archivos en esta categoría
            for archivo in os.listdir(ruta_categoria):
                ruta_archivo = os.path.join(ruta_categoria, archivo)
                
                # Verificamos que sea un archivo
                if os.path.isfile(ruta_archivo):
                    archivos.append(ruta_archivo)
                    categorias.append(categoria)
    
    return archivos, categorias

In [5]:
def eliminar_cabecera_firma(texto):
    """
    Elimina la cabecera y opcionalmente la firma de un documento, usando criterios
    específicos y conservadores para la detección de firmas.
    """
    import re
    
    lineas = texto.split('\n')
    
    # Buscamos la primera línea en blanco que separa la cabecera del cuerpo
    indice_final_cabecera = -1
    for i, linea in enumerate(lineas):
        if linea.strip() == '' and i > 0:  # Aseguramos que no sea la primera línea
            indice_final_cabecera = i
            break
    
    # Si encontramos la línea en blanco, eliminamos la cabecera
    if indice_final_cabecera != -1:
        texto_sin_cabecera = '\n'.join(lineas[indice_final_cabecera + 1:])
    else:
        texto_sin_cabecera = texto
    
    # Eliminación de firmas con criterios más específicos
    lineas = texto_sin_cabecera.split('\n')
    
    # Criterios para detectar inicio de firma
    posibles_inicios_firma = []
    
    for i, linea in enumerate(lineas):
        # Criterio 1: Líneas que son separadores claros
        if linea.strip() in ['--', '---', '----'] or linea.startswith('-- ') or linea.startswith('- '):
            posibles_inicios_firma.append(i)
        
        # Criterio 2: Líneas que contienen solo un correo electrónico
        elif re.match(r'^\s*[\w\.-]+@[\w\.-]+\s*$', linea):
            posibles_inicios_firma.append(i)
    
    # Si encontramos posibles inicios de firma
    if posibles_inicios_firma:
        # Tomamos el primer posible inicio
        indice_inicio_firma = posibles_inicios_firma[0]
        texto_sin_firma = '\n'.join(lineas[:indice_inicio_firma])
    else:
        texto_sin_firma = texto_sin_cabecera
    
    return texto_sin_firma

In [6]:
def leer_y_procesar_documentos(ruta_base):
    """
    Lee los archivos, elimina cabeceras y firmas, y devuelve un DataFrame
    """
    archivos, categorias = listar_archivos(ruta_base)
    
    documentos = []
    textos_originales = []
    textos_procesados = []
    
    for archivo in archivos:
        try:
            with open(archivo, 'r', encoding='utf-8', errors='ignore') as f:
                texto = f.read()
            
            textos_originales.append(texto)
            
            # Eliminamos cabecera y firma
            texto_procesado = eliminar_cabecera_firma(texto)
            textos_procesados.append(texto_procesado)
            
            documentos.append({
                'archivo': archivo,
                'categoria': os.path.basename(os.path.dirname(archivo)),
                'texto_original': texto,
                'texto_procesado': texto_procesado
            })
        except Exception as e:
            print(f"Error al procesar archivo {archivo}: {e}")
    
    return pd.DataFrame(documentos)

In [7]:
def preprocesar_texto(texto, stemming=True):
    """
    Realiza el preprocesamiento completo del texto:
    1. Tokenización
    2. Eliminación de palabras vacías
    3. Stemming o lematización
    """
    try:
        # Tokenización usando el tokenizador ya descargado
        from nltk.tokenize import word_tokenize
        tokens = word_tokenize(texto.lower())
    except LookupError:
        # Si hay algún problema con el tokenizador, usamos una alternativa simple
        texto_lower = texto.lower()
        # Eliminamos puntuación básica
        for char in '.,;:!?"()[]{}/-_':
            texto_lower = texto_lower.replace(char, ' ')
        # Tokenizamos por espacios
        tokens = texto_lower.split()
    
    # Eliminación de signos de puntuación y caracteres especiales
    tokens = [token for token in tokens if token.isalpha()]
    
    # Eliminación de stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming o lematización
    if stemming:
        # Aplicamos stemming (Porter)
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    else:
        # Aplicamos lematización (WordNet)
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

In [8]:
def generar_representacion_tf(documentos_procesados):
    """
    Genera la representación TF para los documentos
    """
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(documentos_procesados)
    
    # Convertimos a DataFrame para mejor visualización
    df_tf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    
    return df_tf, vectorizer

def generar_representacion_tfidf(documentos_procesados):
    """
    Genera la representación TF-IDF para los documentos
    """
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(documentos_procesados)
    
    # Convertimos a DataFrame para mejor visualización
    df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    
    return df_tfidf, vectorizer

In [9]:
def cargar_modelo_word2vec(ruta_modelo=None):
    """
    Carga un modelo Word2Vec preentrenado o crea uno simple si no se proporciona ruta
    """
    if ruta_modelo and os.path.exists(ruta_modelo):
        # Cargar modelo preentrenado desde una ruta específica
        modelo = KeyedVectors.load_word2vec_format(ruta_modelo, binary=True)
    else:
        try:
            # Intentar usar la importación correcta para el downloader
            import gensim.downloader as api
            try:
                print("Intentando cargar 'glove-wiki-gigaword-50'...")
                modelo = api.load('glove-wiki-gigaword-50')  # Modelo más pequeño
            except:
                print("Intentando cargar 'word2vec-google-news-300'...")
                modelo = api.load('word2vec-google-news-300')
        except Exception as e:
            print(f"Error al cargar modelos preentrenados: {e}")
            print("Creando un modelo con los documentos disponibles...")
            
            # Extraemos las listas de tokens de los documentos
            documentos_tokens = df_documentos['tokens'].tolist()
            
            # Entrenamos un modelo Word2Vec con nuestros documentos
            modelo = Word2Vec(sentences=documentos_tokens, vector_size=100, window=5, min_count=1, workers=4)
            
            # Accedemos a los vectores
            modelo = modelo.wv
    
    return modelo

def generar_representacion_aditiva(documentos_tokens, modelo):
    """
    Genera la representación aditiva para los documentos usando un modelo preentrenado
    """
    representaciones = []
    
    for tokens in documentos_tokens:
        # Filtramos solo palabras que están en el vocabulario del modelo
        tokens_validos = [token for token in tokens if token in modelo.key_to_index]
        
        if tokens_validos:
            # Modelo aditivo: sumamos los vectores de todas las palabras
            vector_documento = sum(modelo[token] for token in tokens_validos)
        else:
            # Si no hay tokens válidos, usamos un vector de ceros
            vector_documento = np.zeros(modelo.vector_size)
        
        representaciones.append(vector_documento)
    
    return np.array(representaciones)

def generar_representacion_media(documentos_tokens, modelo):
    """
    Genera la representación media para los documentos usando un modelo preentrenado
    """
    representaciones = []
    
    for tokens in documentos_tokens:
        # Filtramos solo palabras que están en el vocabulario del modelo
        tokens_validos = [token for token in tokens if token in modelo.key_to_index]
        
        if tokens_validos:
            # Modelo de la media: promediamos los vectores de todas las palabras
            vector_documento = sum(modelo[token] for token in tokens_validos) / len(tokens_validos)
        else:
            # Si no hay tokens válidos, usamos un vector de ceros
            vector_documento = np.zeros(modelo.vector_size)
        
        representaciones.append(vector_documento)
    
    return np.array(representaciones)

In [10]:
def guardar_representacion(representacion, nombre_archivo):
    """
    Guarda la representación en un archivo de texto
    """
    print(f"Guardando en {nombre_archivo}...")
    
    if isinstance(representacion, pd.DataFrame):
        # Si es un DataFrame (como TF o TF-IDF)
        representacion.to_csv(nombre_archivo, sep='\t', index=False)
    else:
        # Si es un array (como las representaciones de word embeddings)
        np.savetxt(nombre_archivo, representacion, delimiter='\t')
        
    print(f"Representación guardada en {nombre_archivo}")

In [11]:
def ejecutar_proceso_completo(ruta_base, ruta_salida, usar_stemming=True):
    """
    Ejecuta el proceso completo de generación de representaciones
    """
    print("Leyendo y procesando documentos...")
    df_documentos = leer_y_procesar_documentos(ruta_base)
    
    print(f"Se han procesado {len(df_documentos)} documentos de {df_documentos['categoria'].nunique()} categorías")
    
    # Preprocesamiento para el modelo del espacio vectorial
    print("Realizando preprocesamiento de textos...")
    df_documentos['tokens'] = df_documentos['texto_procesado'].apply(
        lambda x: preprocesar_texto(x, stemming=usar_stemming)
    )
    
    # Unimos los tokens para tener texto preprocesado
    df_documentos['texto_preprocesado'] = df_documentos['tokens'].apply(lambda x: ' '.join(x))
    
    # Generamos representaciones para el modelo del espacio vectorial
    print("Generando representación TF...")
    df_tf, _ = generar_representacion_tf(df_documentos['texto_preprocesado'])
    
    print("Generando representación TF-IDF...")
    df_tfidf, _ = generar_representacion_tfidf(df_documentos['texto_preprocesado'])
    
    # Cargamos modelo preentrenado para el modelo semántico vectorial
    print("Cargando modelo Word2Vec preentrenado...")
    modelo_w2v = cargar_modelo_word2vec()
    
    print("Generando representación aditiva...")
    representacion_aditiva = generar_representacion_aditiva(df_documentos['tokens'], modelo_w2v)
    
    print("Generando representación media...")
    representacion_media = generar_representacion_media(df_documentos['tokens'], modelo_w2v)
    
    # Guardamos las representaciones
    print("Guardando representaciones...")
    guardar_representacion(df_tf, os.path.join(ruta_salida, 'representacion_tf.txt'))
    guardar_representacion(df_tfidf, os.path.join(ruta_salida, 'representacion_tfidf.txt'))
    guardar_representacion(representacion_aditiva, os.path.join(ruta_salida, 'representacion_aditiva.txt'))
    guardar_representacion(representacion_media, os.path.join(ruta_salida, 'representacion_media.txt'))
    
    print("Proceso completado correctamente.")
    
    return df_documentos, df_tf, df_tfidf, representacion_aditiva, representacion_media

In [12]:
# Definimos las rutas de entrada y salida
ruta_base = './Corpus-representacion'  # Carpeta con la ubicación de tus datos
ruta_salida = './resultados'  # Carpeta donde se guardarán los resultados

# Creamos la carpeta de salida si no existe
if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Ejecutamos el proceso completo
df_documentos, df_tf, df_tfidf, rep_aditiva, rep_media = ejecutar_proceso_completo(ruta_base, ruta_salida)

Leyendo y procesando documentos...
Se han procesado 805 documentos de 7 categorías
Realizando preprocesamiento de textos...
Generando representación TF...
Generando representación TF-IDF...
Cargando modelo Word2Vec preentrenado...
Intentando cargar 'glove-wiki-gigaword-50'...
[=================---------------------------------] 35.5% 23.4/66.0MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Generando representación aditiva...
Generando representación media...
Guardando representaciones...
Guardando en ./resultados/representacion_tf.txt...
Representación guardada en ./resultados/representacion_tf.txt
Guardando en ./resultados/representacion_tfidf.txt...
Representación guardada en ./resultados/representacion_tfidf.txt
Guardando en ./resultados/representacion_aditiva.txt...
Representación guardada en ./resultados/representacion_aditiva.txt
Guardando en ./resultados/representacion_media.txt...
Representación guardada en ./resultados/representacion_media.txt
Proceso completado correctamente.


In [13]:
def analizar_evolucion_documento(df_documentos, indice=0):
    """
    Analiza la evolución de un documento a lo largo del procesamiento,
    con mejor visualización de la firma
    """
    documento = df_documentos.iloc[indice]
    
    print(f"Documento de la categoría: {documento['categoria']}")
    print("\n----- TEXTO ORIGINAL (primeras 300 caracteres) -----")
    print(documento['texto_original'][:300])
    
    print("\n----- TEXTO SIN CABECERA NI FIRMA (primeras 300 caracteres) -----")
    print(documento['texto_procesado'][:300])
    
    # Identificar mejor la posible firma
    lineas_originales = documento['texto_original'].split('\n')
    texto_sin_cabecera = None
    
    # Identificar el final de la cabecera (primera línea en blanco)
    for i, linea in enumerate(lineas_originales):
        if linea.strip() == '' and i > 0:
            texto_sin_cabecera = '\n'.join(lineas_originales[i+1:])
            break
    
    if texto_sin_cabecera:
        lineas_sin_cabecera = texto_sin_cabecera.split('\n')
        
        # Buscar posibles inicios de firma
        posibles_inicios = []
        for i, linea in enumerate(lineas_sin_cabecera):
            if (linea.strip() in ['--', '---', '----'] or 
                linea.startswith('-- \n') or 
                ('@' in linea and len(linea.split()) <= 3)):
                posibles_inicios.append(i)
        
        if posibles_inicios:
            indice_firma = posibles_inicios[0]
            print("\n----- POSIBLE FIRMA DETECTADA -----")
            print('\n'.join(lineas_sin_cabecera[indice_firma:]))
        else:
            print("\n----- NO SE DETECTÓ UNA FIRMA CLARA -----")
    else:
        print("\n----- NO SE PUDO IDENTIFICAR LA CABECERA -----")
    
    # También mostramos final del texto procesado
    print("\n----- FINAL DEL TEXTO PROCESADO (últimas 5 líneas) -----")
    lineas_procesadas = documento['texto_procesado'].split('\n')
    for linea in lineas_procesadas[-5:]:
        print(linea)
    
    print("\n----- TOKENS DESPUÉS DEL PREPROCESAMIENTO (primeros 36) -----")
    print(documento['tokens'][:36])
    
    print("\n----- TEXTO PREPROCESADO (primeros 100 caracteres) -----")
    print(documento['texto_preprocesado'][:100])
    
    # Análisis cuantitativo
    print("\n----- ANÁLISIS CUANTITATIVO -----")
    print(f"Longitud del texto original: {len(documento['texto_original'])} caracteres")
    print(f"Longitud del texto sin cabecera ni firma: {len(documento['texto_procesado'])} caracteres")
    print(f"Número de tokens después del preprocesamiento: {len(documento['tokens'])}")
    
    # Calculamos la reducción en porcentaje
    reduccion_cabecera = 100 * (1 - len(documento['texto_procesado']) / len(documento['texto_original']))
    reduccion_preprocesamiento = 100 * (1 - len(documento['tokens']) / len(documento['texto_procesado'].split()))
    
    print(f"Reducción después de eliminar cabecera y firma: {reduccion_cabecera:.2f}%")
    print(f"Reducción después del preprocesamiento: {reduccion_preprocesamiento:.2f}%")

In [14]:
# Analizamos la evolución de un documento ejemplo aleatoriamente
import random
indice_aleatorio = random.randint(0, len(df_documentos) - 1)
print(f"DOCUMENTO ALEATORIO (índice {indice_aleatorio}):")
analizar_evolucion_documento(df_documentos, indice=indice_aleatorio)

DOCUMENTO ALEATORIO (índice 654):
Documento de la categoría: sci.electronics

----- TEXTO ORIGINAL (primeras 300 caracteres) -----
Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!europa.eng.gtefsd.com!howland.reston.ans.net!noc.near.net!uunet!pipex!uknet!uknet!cam-eng!cmh
From: cmh@eng.cam.ac.uk (C.M. Hicks)
Newsgroups: sci.electronics
Subject: Re: MICROPHONE PRE-AMP/LOW NOISE/PHANTOM POWERED
Message-ID: <1993Apr16.090648

----- TEXTO SIN CABECERA NI FIRMA (primeras 300 caracteres) -----
davidj@rahul.net (David Josephson) writes:

>In <C5JJJ2.1tF@cmcl2.nyu.edu> ali@cns.nyu.edu (Alan Macaluso) writes:

>>I'm looking to build a microphone preamp that has very good low-noise characteristics,  large clean gain, and incorportates phantom power (20-48 volts (dc)) for a PZM microphone.  I'

----- POSIBLE FIRMA DETECTADA -----
--
  Christopher Hicks    |      Paradise is a Linear Gaussian World
  cmh@uk.ac.cam.eng    |    (also reported to taste hot and sweaty)


----- FINAL DEL TEXTO PR

In [15]:
# Analizamos la evolución de un documento ejemplo (79) para el informe
print(f"DOCUMENTO ALEATORIO (índice {79}):")
analizar_evolucion_documento(df_documentos, indice=79)

DOCUMENTO ALEATORIO (índice 79):
Documento de la categoría: rec.autos

----- TEXTO ORIGINAL (primeras 300 caracteres) -----
Newsgroups: rec.autos
Path: cantaloupe.srv.cs.cmu.edu!rochester!cornell!batcomputer!caen!spool.mu.edu!torn!watserv2.uwaterloo.ca!watserv1!mks.com!mike
From: mike@mks.com (Mike Brookbank)
Subject: MGBs and the real world
Message-ID: <1993Apr5.181056.29411@mks.com>
Organization: Mortice Kern Systems I

----- TEXTO SIN CABECERA NI FIRMA (primeras 300 caracteres) -----
My sister has an MGB.  She has one from the last year they were produced
(1978? 1979?).  Its in very good shape.  I've been bugging her for years
about selling it.  I've said over and over that she should sell it
before the car is worthless while she maintains that the car may
actually be increasing

----- POSIBLE FIRMA DETECTADA -----
-- 
------------------------------------------------------------------------------
Mike Brookbank,                 |MKS| 35 King St. North       mike@mks.com 
Director, I

In [16]:
def generar_y_guardar_visualizaciones(df_documentos, df_tf, df_tfidf, rep_aditiva, rep_media, ruta_salida):
    """
    Genera y guarda visualizaciones adicionales para el análisis de las representaciones
    """
    import os
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    from sklearn.decomposition import PCA
    from sklearn.manifold import TSNE
    from sklearn.metrics.pairwise import cosine_similarity
    
    # Crear la carpeta para gráficas si no existe
    ruta_graficas = os.path.join(ruta_salida, 'graficas')
    if not os.path.exists(ruta_graficas):
        os.makedirs(ruta_graficas)
    
    # 1. Longitud media de documentos por categoría

    # Calculamos los datos para el gráfico
    datos_longitud = df_documentos.groupby('categoria')['tokens'].apply(lambda x: np.mean([len(tokens) for tokens in x]))
    
    # Función para personalizar nombres de categorías
    def acortar_categoria(categoria, max_len=18):
        # Acortamos categorías específicas según reglas personalizadas
        if len(categoria) > max_len:
            return categoria[:max_len-3] + "..."
        else:
            return categoria
    
    plt.figure(figsize=(10, 6))
    valores = datos_longitud.values
    categorias = [acortar_categoria(cat) for cat in datos_longitud.index]
    plt.bar(range(len(valores)), valores)
    plt.xticks(range(len(categorias)), categorias, rotation=0, fontsize=9)
    plt.ylabel('Número medio de tokens')
    plt.xlabel('')
    plt.tight_layout()
    plt.savefig(os.path.join(ruta_graficas, 'longitud_por_categoria.png'))
    plt.close()

    # 2. Palabras más frecuentes
    todas_palabras = [token for tokens in df_documentos['tokens'] for token in tokens]
    contador = Counter(todas_palabras)
    palabras_comunes = contador.most_common(20)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x=[palabra for palabra, _ in palabras_comunes], 
                y=[frecuencia for _, frecuencia in palabras_comunes])
    # plt.title('20 palabras más frecuentes en la colección')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(ruta_graficas, 'palabras_frecuentes.png'))
    plt.close()
    
    # 3. Dimensionalidad de las representaciones
    dimensiones = {
        'TF/TF-IDF': len(df_tf.columns),
        'Word Embeddings': rep_aditiva.shape[1]
    }
    
    plt.figure(figsize=(8, 5))
    plt.bar(dimensiones.keys(), dimensiones.values())
    # plt.title('Dimensionalidad de las representaciones')
    plt.ylabel('Número de dimensiones')
    plt.tight_layout()
    plt.savefig(os.path.join(ruta_graficas, 'dimensionalidad.png'))
    plt.close()
    
    # 4. NUEVA: Dispersión de las representaciones
    # Calculamos el porcentaje de ceros en TF y TF-IDF
    sparsity_tf = (df_tf.values == 0).sum() / df_tf.size * 100
    sparsity_tfidf = (df_tfidf.values == 0).sum() / df_tfidf.size * 100
    sparsity_embed = (rep_aditiva == 0).sum() / rep_aditiva.size * 100
    
    plt.figure(figsize=(8, 5))
    plt.bar(['TF', 'TF-IDF', 'Word Embeddings'], [sparsity_tf, sparsity_tfidf, sparsity_embed])
    # plt.title('Dispersión de las representaciones')
    plt.ylabel('Porcentaje de valores cero (%)')
    plt.tight_layout()
    plt.savefig(os.path.join(ruta_graficas, 'dispersion.png'))
    plt.close()
    
    # 5. NUEVA: Visualización de agrupamiento con PCA
    # Seleccionamos un subconjunto para visualización (para mayor rapidez)
    muestra = min(500, len(df_documentos))
    np.random.seed(42)
    indices = np.random.choice(len(df_documentos), muestra, replace=False)
    
    # Aplicamos PCA a las diferentes representaciones
    try:
        # Para TF-IDF
        pca_tfidf = PCA(n_components=2, random_state=42).fit_transform(df_tfidf.iloc[indices].values)
        
        # Para embeddings aditivos
        pca_embed = PCA(n_components=2, random_state=42).fit_transform(rep_aditiva[indices])
        
        # Categorías para colorear
        categorias = df_documentos.iloc[indices]['categoria'].values
        
        # Visualización de PCA para TF-IDF
        plt.figure(figsize=(10, 8))
        for categoria in np.unique(categorias):
            indices_cat = [i for i, cat in enumerate(categorias) if cat == categoria]
            plt.scatter(pca_tfidf[indices_cat, 0], pca_tfidf[indices_cat, 1], label=categoria, alpha=0.7)
        #plt.title('Visualización PCA de representaciones TF-IDF')
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(ruta_graficas, 'pca_tfidf.png'))
        plt.close()
        
        # Visualización de PCA para embeddings
        plt.figure(figsize=(10, 8))
        for categoria in np.unique(categorias):
            indices_cat = [i for i, cat in enumerate(categorias) if cat == categoria]
            plt.scatter(pca_embed[indices_cat, 0], pca_embed[indices_cat, 1], label=categoria, alpha=0.7)
        # plt.title('Visualización PCA de representaciones de Word Embeddings')
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(ruta_graficas, 'pca_embeddings.png'))
        plt.close()
    except Exception as e:
        print(f"Error al generar visualizaciones PCA: {e}")
    
    # 6. NUEVA: Matriz de similaridad para documentos de ejemplo
    try:
        # Seleccionamos algunos documentos de ejemplo (uno de cada categoría)
        categorias_unicas = df_documentos['categoria'].unique()
        indices_ejemplo = [df_documentos[df_documentos['categoria'] == cat].index[0] for cat in categorias_unicas]
        
        # Calculamos similaridad coseno entre estos documentos usando TF-IDF
        sim_tfidf = cosine_similarity(df_tfidf.iloc[indices_ejemplo])
        
        # Calculamos similaridad coseno usando embeddings
        sim_embed = cosine_similarity(rep_aditiva[indices_ejemplo])
        
        # Visualizamos las matrices de similaridad
        plt.figure(figsize=(10, 8))
        sns.heatmap(sim_tfidf, annot=True, cmap='Blues', xticklabels=categorias_unicas, yticklabels=categorias_unicas)
        # plt.title('Similaridad coseno entre documentos (TF-IDF)')
        plt.tight_layout()
        plt.savefig(os.path.join(ruta_graficas, 'similaridad_tfidf.png'))
        plt.close()
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(sim_embed, annot=True, cmap='Blues', xticklabels=categorias_unicas, yticklabels=categorias_unicas)
        # plt.title('Similaridad coseno entre documentos (Word Embeddings)')
        plt.tight_layout()
        plt.savefig(os.path.join(ruta_graficas, 'similaridad_embeddings.png'))
        plt.close()
    except Exception as e:
        print(f"Error al generar matrices de similaridad: {e}")
    
    print(f"Visualizaciones guardadas en {ruta_graficas}")

In [17]:
# Después de ejecutar el proceso completo y obtener las representaciones
df_documentos, df_tf, df_tfidf, rep_aditiva, rep_media = ejecutar_proceso_completo(ruta_base, ruta_salida)

# Generar y guardar visualizaciones
generar_y_guardar_visualizaciones(df_documentos, df_tf, df_tfidf, rep_aditiva, rep_media, ruta_salida)

Leyendo y procesando documentos...
Se han procesado 805 documentos de 7 categorías
Realizando preprocesamiento de textos...
Generando representación TF...
Generando representación TF-IDF...
Cargando modelo Word2Vec preentrenado...
Intentando cargar 'glove-wiki-gigaword-50'...
Generando representación aditiva...
Generando representación media...
Guardando representaciones...
Guardando en ./resultados/representacion_tf.txt...
Representación guardada en ./resultados/representacion_tf.txt
Guardando en ./resultados/representacion_tfidf.txt...
Representación guardada en ./resultados/representacion_tfidf.txt
Guardando en ./resultados/representacion_aditiva.txt...
Representación guardada en ./resultados/representacion_aditiva.txt
Guardando en ./resultados/representacion_media.txt...
Representación guardada en ./resultados/representacion_media.txt
Proceso completado correctamente.
Visualizaciones guardadas en ./resultados/graficas


In [18]:
# Análisis complementario: impacto de la lematización en modelos semánticos vectoriales
print("Análisis complementario: comparación entre truncamiento (stemming) y lematización")

def preprocesar_texto_con_lematizacion(texto):
    """
    Realiza el preprocesamiento completo del texto usando lematización en lugar de stemming:
    1. Tokenización
    2. Eliminación de palabras vacías
    3. Lematización
    """
    try:
        # Tokenización usando el tokenizador NLTK
        tokens = word_tokenize(texto.lower())
    except LookupError:
        # Si hay algún problema con el tokenizador, usamos una alternativa simple
        texto_lower = texto.lower()
        # Eliminamos puntuación básica
        for char in '.,;:!?"()[]{}/-_':
            texto_lower = texto_lower.replace(char, ' ')
        # Tokenizamos por espacios
        tokens = texto_lower.split()
    
    # Eliminación de signos de puntuación y caracteres especiales
    tokens = [token for token in tokens if token.isalpha()]
    
    # Eliminación de stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Aplicamos lematización en lugar de stemming
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

Análisis complementario: comparación entre truncamiento (stemming) y lematización


In [19]:
# Comparamos el vocabulario resultante con stemming vs lematización
def comparar_vocabularios(df_documentos):
    """
    Compara el vocabulario resultante usando stemming vs lematización
    """
    # Procesamos los documentos con lematización
    print("Procesando documentos con lematización...")
    df_documentos['tokens_lematizados'] = df_documentos['texto_procesado'].apply(
        lambda x: preprocesar_texto_con_lematizacion(x)
    )
    
    # Obtenemos los vocabularios
    vocab_stemming = set(token for tokens in df_documentos['tokens'] for token in tokens)
    vocab_lematizacion = set(token for tokens in df_documentos['tokens_lematizados'] for token in tokens)
    
    # Calculamos estadísticas
    print(f"Tamaño del vocabulario con stemming: {len(vocab_stemming)}")
    print(f"Tamaño del vocabulario con lematización: {len(vocab_lematizacion)}")
    
    # Intersección y diferencias
    interseccion = vocab_stemming.intersection(vocab_lematizacion)
    solo_stemming = vocab_stemming - vocab_lematizacion
    solo_lematizacion = vocab_lematizacion - vocab_stemming
    
    print(f"Términos comunes a ambos vocabularios: {len(interseccion)} ({len(interseccion)/len(vocab_stemming)*100:.1f}% del vocabulario con stemming)")
    print(f"Términos únicos de stemming: {len(solo_stemming)}")
    print(f"Términos únicos de lematización: {len(solo_lematizacion)}")
    
    # Mostramos algunos ejemplos de términos diferentes
    print("\nEjemplos de términos con stemming que no aparecen en lematización:")
    for term in list(solo_stemming)[:10]:
        print(f"  - {term}")
    
    print("\nEjemplos de términos con lematización que no aparecen en stemming:")
    for term in list(solo_lematizacion)[:10]:
        print(f"  - {term}")
    
    return vocab_stemming, vocab_lematizacion, interseccion

In [20]:
# Generamos representaciones basadas en word embeddings con lematización
def generar_representaciones_con_lematizacion(df_documentos, modelo_w2v):
    """
    Genera representaciones aditiva y media usando lematización
    """
    # Generamos la representación aditiva
    print("Generando representación aditiva con lematización...")
    representacion_aditiva_lem = generar_representacion_aditiva(df_documentos['tokens_lematizados'], modelo_w2v)
    
    # Generamos la representación media
    print("Generando representación media con lematización...")
    representacion_media_lem = generar_representacion_media(df_documentos['tokens_lematizados'], modelo_w2v)
    
    # Guardamos las representaciones
    print("Guardando representaciones...")
    guardar_representacion(representacion_aditiva_lem, os.path.join(ruta_salida, 'representacion_aditiva_lematizacion.txt'))
    guardar_representacion(representacion_media_lem, os.path.join(ruta_salida, 'representacion_media_lematizacion.txt'))
    
    return representacion_aditiva_lem, representacion_media_lem

In [21]:
# Analizamos la cobertura léxica en el modelo preentrenado
def analizar_cobertura_lexica(df_documentos, modelo_w2v):
    """
    Analiza cuántos tokens del vocabulario están presentes en el modelo preentrenado
    """
    # Cobertura con stemming
    tokens_stemming = [token for tokens in df_documentos['tokens'] for token in tokens]
    tokens_unicos_stemming = set(tokens_stemming)
    tokens_en_modelo_stemming = sum(1 for token in tokens_unicos_stemming if token in modelo_w2v.key_to_index)
    
    # Cobertura con lematización
    tokens_lematizacion = [token for tokens in df_documentos['tokens_lematizados'] for token in tokens]
    tokens_unicos_lematizacion = set(tokens_lematizacion)
    tokens_en_modelo_lematizacion = sum(1 for token in tokens_unicos_lematizacion if token in modelo_w2v.key_to_index)
    
    # Calculamos porcentajes
    cobertura_stemming = tokens_en_modelo_stemming / len(tokens_unicos_stemming) * 100
    cobertura_lematizacion = tokens_en_modelo_lematizacion / len(tokens_unicos_lematizacion) * 100
    
    print(f"Cobertura léxica con stemming: {tokens_en_modelo_stemming}/{len(tokens_unicos_stemming)} ({cobertura_stemming:.2f}%)")
    print(f"Cobertura léxica con lematización: {tokens_en_modelo_lematizacion}/{len(tokens_unicos_lematizacion)} ({cobertura_lematizacion:.2f}%)")
    print(f"Mejora en cobertura léxica: {cobertura_lematizacion - cobertura_stemming:.2f} puntos porcentuales")
    
    return cobertura_stemming, cobertura_lematizacion

In [22]:
# Ejecutamos el análisis comparativo
vocab_stemming, vocab_lematizacion, interseccion = comparar_vocabularios(df_documentos)

# Cargamos modelo preentrenado
print("\nCargando modelo Word2Vec preentrenado...")
modelo_w2v = cargar_modelo_word2vec()

# Analizamos la cobertura léxica
print("\nAnalizando cobertura léxica en el modelo preentrenado...")
cobertura_stemming, cobertura_lematizacion = analizar_cobertura_lexica(df_documentos, modelo_w2v)

# Generamos las nuevas representaciones con lematización
representacion_aditiva_lem, representacion_media_lem = generar_representaciones_con_lematizacion(df_documentos, modelo_w2v)

print("\nAnálisis complementario completado. Las nuevas representaciones basadas en lematización han sido guardadas.")

Procesando documentos con lematización...
Tamaño del vocabulario con stemming: 10044
Tamaño del vocabulario con lematización: 12553
Términos comunes a ambos vocabularios: 6518 (64.9% del vocabulario con stemming)
Términos únicos de stemming: 3526
Términos únicos de lematización: 6035

Ejemplos de términos con stemming que no aparecen en lematización:
  - overtak
  - transmitt
  - gover
  - tennesse
  - upstair
  - roommat
  - makalel
  - machin
  - doubl
  - uncontitut

Ejemplos de términos con lematización que no aparecen en stemming:
  - semitic
  - rising
  - hades
  - adobe
  - federation
  - symmetry
  - pas
  - caput
  - dumped
  - fairy

Cargando modelo Word2Vec preentrenado...
Intentando cargar 'glove-wiki-gigaword-50'...

Analizando cobertura léxica en el modelo preentrenado...
Cobertura léxica con stemming: 6710/10044 (66.81%)
Cobertura léxica con lematización: 11185/12553 (89.10%)
Mejora en cobertura léxica: 22.30 puntos porcentuales
Generando representación aditiva con lema